In [ ]:
import pickle 
import networkx as nx
import random
import matplotlib.pyplot as plt
import numpy as np
from networkx.utils import not_implemented_for
from networkx.utils import py_random_state

# A function to generate Lambiotte model  
def Lambiotte_model(m0,q,N):
    G=nx.complete_graph(m0)  # Initial empty graph         
    G.add_node(0)  # Add node 0 to the empty network as the initial node.  
                 
    for source in range(m0, N): # Start network formation from m0 node and stop at N node
        #################################################################################################################                    
        # Step1. Pick a node randomly and make connection.
        # ########################################################################################################################                                                                   
        nodes = [nod for nod in G.nodes()] # Existing nodes in the network
        node = random.choice(nodes) # Choose a random target node
        neighbours = [nbr for nbr in G.neighbors(node) # neighbors of the node
                          if not G.has_edge(source, nbr) # If no edge btn source node and nbr node
                          and not nbr == source] # If neighbor node is not equal to source node 
        G.add_node(source) ## Add node to the network # SOURCE MUST BE ADDED AFTER RANDOM SAMPLING SO AS 
                           ## TO AVOID SELF LOOP WHEN ADDING EDGES
        G.add_edge(source, node) # Add edge 
        
        #################################################################################################################                    
        # Step2. With prob q, make connection with all nbrs.
        #########################################################################################################################                                                                             
        while len(neighbours)>0: # all neighbours need to make connection with the new node
           nbr = neighbours.pop() #
           if q >random.random():  
              G.add_edge(source, nbr) # Add edge 
    return G 

In [ ]:
# Add code for FOF model
def FOF_model(n_q,q,N): 
    '''
    n_q : Number of neighbours
    q : Probability of a new node to attach to neighbouring nodes
    '''  
    # Initializing the network
    m1= n_q + 2 # Initial number of nodes
    G = nx.complete_graph(m1)  # Initial graph
    
            # Growth of the network
    for source in range(m1, N): # Start connection from m0 node and stop at N
        #################################################################################################################                    
        # Step1. Pick one node randomly and make connection.
        # ########################################################################################################################                                                           
        existing_nodes = [nod for nod in G.nodes()]
        target_node = random.choice(existing_nodes)

        neighbours = [nbr for nbr in G.neighbors(target_node)]     
       
        G.add_node(source) ## Adding node to the network # SOURCE MUST BE ADDED AFTER RANDOM SAMPLING SO AS 
                            ## TO AVOID SELF LOOP WHEN ADDING EDGES
        G.add_edge(source, target_node) # Add edge 
    
        ################################################################################################################                    
        ##### Step2. Independently link the target node with each of n_q neighbors with probability q
                     # Pick n_q nodes randomly without replacement and with prob q, make connection.
        ########################################################################################################################                                                                               
        num_neighbours = random.sample(neighbours, min(n_q, len(neighbours))) # Random neighbors of the target node
        for neighbor in num_neighbours:
            if random.random() <=q:
                G.add_edge(source, neighbor)
    return G 

In [ ]:
@py_random_state(2)
@not_implemented_for('directed') 
def node_clustering(G, trials=1000, seed=None):
    #Calculates  [trials] weakly connected triangle clusters in graph G
    n = len(G)
    triangles = 0
    nodes = list(G)
    
    #List of all degree, triangle pairs
    degree_and_triangles=[]
    
    
    for i in [int(seed.random() * n) for i in range(trials)]:
        #Can take same node twice or more. Can be a problem on small graphs.
        nbrs = list(G[nodes[i]])
        degree=len(nbrs)
        
        triangles=0
        
#            
        for j,u in enumerate(nbrs):
            #Find all pairs
            for v in nbrs[j+1:]:
                if (u in G[v]):
                    triangles += 1
        if degree > 1:
            degree_and_triangles.append([degree,(triangles/(degree*(degree-1)/2))])  
        else:
            degree_and_triangles.append([degree,0])
            
    return degree_and_triangles 

In [ ]:
# Comparing the clustering of Lambiotte et al networks with different q's

N = 1000# Network size # population size

# simulate the networks
Network= Lambiotte_model(3, 0.1, N)
Network1= Lambiotte_model(3, 0.3, N)
Network2= Lambiotte_model(3, 0.5, N)
Network3= Lambiotte_model(3, 0.8, N)

# compute degree and clustering
Degrees_and_clustering_coefficients =node_clustering(Network, N)
Degrees_and_clustering_coefficients_1 =node_clustering(Network1, N)
Degrees_and_clustering_coefficients_2 =node_clustering(Network2, N)
Degrees_and_clustering_coefficients_3 =node_clustering(Network3, N)

array_DC, array_DC_1 = np.array(Degrees_and_clustering_coefficients), np.array(Degrees_and_clustering_coefficients_1)
array_DC_2, array_DC_3 = np.array(Degrees_and_clustering_coefficients_2), np.array(Degrees_and_clustering_coefficients_3)

CC, CC_1, CC_2, CC_3 = array_DC[:,1], array_DC_1[:,1], array_DC_2[:,1], array_DC_3[:,1]
degree, degree_1, degree_2, degree_3 = array_DC[:,0], array_DC_1[:,0], array_DC_2[:,0], array_DC_3[:,0]


In [ ]:
#Find nodes with degree k and calculate their mean
max_k, max_k_1, max_k_2, max_k_3 = max(array_DC[:,0]), max(array_DC_1[:,0]), max(array_DC_2[:,0]), max(array_DC_3[:,0])

meancluster, meancluster1 = np.zeros(int(max_k)), np.zeros(int(max_k_1))
meancluster2, meancluster3 = np.zeros(int(max_k_2)), np.zeros(int(max_k_3))

for k in np.arange(max_k):
    #Find nodes with degree k
    withdegreek=array_DC[:,0]==k
    meancluster[int(k)]=np.mean(array_DC[withdegreek,1])
    

for k in np.arange(max_k_1):
    #Find nodes with degree k
    withdegreek1=array_DC_1[:,0]==k
    meancluster1[int(k)]=np.mean(array_DC_1[withdegreek1,1])

for k in np.arange(max_k_2):
    #Find nodes with degree k
    withdegreek2=array_DC_2[:,0]==k
    meancluster2[int(k)]=np.mean(array_DC_2[withdegreek2,1])

for k in np.arange(max_k_3):
    #Find nodes with degree k
    withdegreek3=array_DC_3[:,0]==k
    meancluster3[int(k)]=np.mean(array_DC_3[withdegreek3,1])



In [ ]:
# plot the clustering against the degree
mk = max(max_k, max_k_1, max_k_2, max_k_3)

fig, ax = plt.subplots(1, figsize=(8, 6), sharex=True, sharey=True, squeeze= True)    
ax.scatter(np.arange(max_k), meancluster,color= 'cyan',linestyle = '--')
ax.scatter(np.arange(max_k_1), meancluster1,color= 'blue',linestyle = '--')
ax.scatter(np.arange(max_k_2), meancluster2,color= 'hotpink',linestyle = '--')
ax.scatter(np.arange(max_k_3), meancluster3,color= 'red',linestyle = '--')


plt.ylabel("$C_{k}$", fontsize=12)  
plt.xlabel("Degree $k$", fontsize=12)  
#plt.xlim([2,max_k])  # Limiting x axis
plt.rcParams['font.size'] = '20'

In [ ]:
# simulate FOF with q=0.3 and varying q
# simulate the networks
FNetwork= FOF_model(3, 0.3, N)
FNetwork1= FOF_model(5, 0.3, N)
FNetwork2= FOF_model(3, 0.8, N)
FNetwork3= FOF_model(5, 0.8, N)

# compute degree and clustering
FDegrees_and_clustering_coefficients =node_clustering(FNetwork, N)
FDegrees_and_clustering_coefficients_1 =node_clustering(FNetwork1, N)
FDegrees_and_clustering_coefficients_2 =node_clustering(FNetwork2, N)
FDegrees_and_clustering_coefficients_3 =node_clustering(FNetwork3, N)

Farray_DC, Farray_DC_1 = np.array(FDegrees_and_clustering_coefficients), np.array(FDegrees_and_clustering_coefficients_1)
Farray_DC_2, Farray_DC_3 = np.array(FDegrees_and_clustering_coefficients_2), np.array(FDegrees_and_clustering_coefficients_3)

FCC, FCC_1, FCC_2, FCC_3 = Farray_DC[:,1], Farray_DC_1[:,1], Farray_DC_2[:,1], Farray_DC_3[:,1]
Fdegree, Fdegree_1, Fdegree_2, Fdegree_3 = Farray_DC[:,0], Farray_DC_1[:,0], Farray_DC_2[:,0], Farray_DC_3[:,0]

In [ ]:
#Find nodes with degree k and calculate their mean
Fmax_k, Fmax_k_1, Fmax_k_2, Fmax_k_3 = max(Farray_DC[:,0]), max(Farray_DC_1[:,0]), max(Farray_DC_2[:,0]), max(Farray_DC_3[:,0])

Fmeancluster, Fmeancluster1 = np.zeros(int(Fmax_k)), np.zeros(int(Fmax_k_1))
Fmeancluster2, Fmeancluster3 = np.zeros(int(Fmax_k_2)), np.zeros(int(Fmax_k_3))

for k in np.arange(Fmax_k):
    #Find nodes with degree k
    Fwithdegreek=Farray_DC[:,0]==k
    Fmeancluster[int(k)]=np.mean(Farray_DC[Fwithdegreek,1])

for k in np.arange(Fmax_k_1):
    #Find nodes with degree k
    Fwithdegreek1=Farray_DC_1[:,0]==k
    Fmeancluster1[int(k)]=np.mean(Farray_DC_1[Fwithdegreek1,1])

for k in np.arange(Fmax_k_2):
    #Find nodes with degree k
    Fwithdegreek2=Farray_DC_2[:,0]==k
    Fmeancluster2[int(k)]=np.mean(Farray_DC_2[Fwithdegreek2,1])

for k in np.arange(Fmax_k_3):
    #Find nodes with degree k
    Fwithdegreek3=Farray_DC_3[:,0]==k
    Fmeancluster3[int(k)]=np.mean(Farray_DC_3[Fwithdegreek3,1])



In [ ]:
#mk = max(max_k, max_k_1, max_k_2, max_k_3)

fig, ax = plt.subplots(1, figsize=(8, 6), sharex=True, sharey=True, squeeze= True)    
ax.scatter(np.arange(Fmax_k), Fmeancluster,color= 'cyan',linestyle = '--')
ax.scatter(np.arange(Fmax_k_1), Fmeancluster1,color= 'blue',linestyle = '--')
ax.scatter(np.arange(Fmax_k_2), Fmeancluster2,color= 'hotpink',linestyle = '--')
ax.scatter(np.arange(Fmax_k_3), Fmeancluster3,color= 'red',linestyle = '--')


plt.ylabel("$C_{k}$", fontsize=12)  
plt.xlabel("Degree $k$", fontsize=12)  
#plt.xlim([2,max_k])  # Limiting x axis
plt.rcParams['font.size'] = '20'

In [ ]:
# Compare Lambiotte and FOF models with fixed n_q, with q=0.3

fig, ax = plt.subplots(1, figsize=(8, 6), sharex=True, sharey=True, squeeze= True)    
ax.scatter(np.arange(max_k_1), meancluster1,color= 'cyan',linestyle = '--')
ax.scatter(np.arange(Fmax_k), Fmeancluster,color= 'blue',linestyle = '--')
ax.scatter(np.arange(Fmax_k_1), Fmeancluster1,color= 'hotpink',linestyle = '--')


plt.ylabel("$C_{k}$", fontsize=12)  
plt.xlabel("Degree $k$", fontsize=12)  
#plt.xlim([2,max_k])  # Limiting x axis
plt.rcParams['font.size'] = '20'

In [ ]:
# Compare Lambiotte and FOF models with fixed n_q, with q=0.8

fig, ax = plt.subplots(1, figsize=(8, 6), sharex=True, sharey=True, squeeze= True)    
ax.scatter(np.arange(max_k_3), meancluster3,color= 'cyan',linestyle = '--')
ax.scatter(np.arange(Fmax_k_2), Fmeancluster2,color= 'blue',linestyle = '--')
ax.scatter(np.arange(Fmax_k_3), Fmeancluster3,color= 'hotpink',linestyle = '--')


plt.ylabel("$C_{k}$", fontsize=12)  
plt.xlabel("Degree $k$", fontsize=12)  
#plt.xlim([2,max_k])  # Limiting x axis
plt.rcParams['font.size'] = '20'